In [1]:
import os
import pandas as pd
import numpy as np
from uuid import uuid4
from bson import Binary
import pickle

In [3]:
n,m = 5000, 50
frame = pd.DataFrame(data=np.random.randn(n, m), columns=[str(uuid4()) for _ in range(0, m)])
frame

,8cea96c4-e085-43b3-94ab-6b782635a741,87cf973c-b031-467b-9317-5ad8a5280469,d57ce7a6-68f9-420f-970a-66b63102cc0c,124418b0-8b5a-40c8-a0c8-5e491cab584b,d5ab6005-1245-47ea-8a7b-52ef2acdb070,98a5a4b1-d720-435d-8cbf-cc8090648044,de552b21-a7dc-4563-9777-b943d9456843,d4ee0282-a49d-4d5d-b1af-e392948b0a8f,0eabeedb-c544-4458-b6c5-74d7b1674ca7,fd33a065-cefc-492e-9ace-84c3d96f6ffb,...,78caa904-26fc-4bfc-9a34-ca54039aec35,6f4c7fec-b506-498a-b0bb-93b083a3eb0a,05ff5700-e575-46e2-a947-9e938a678657,00c14493-9c3b-4cb9-9116-a982e2c76d66,99a4a587-2d88-4418-9125-5730a63da3c2,a3c85bb9-63a2-4913-89d9-a0d2c74de651,f46c0dfb-c293-4bc3-823e-9c0a554e85e6,376cf1e7-626f-4f5a-ab5f-1eb27ac2df43,4d57afc9-5de4-495e-8557-bbd15a978eb3,9569e913-070d-479e-a720-69b0c201b59a
0,0.053723,-0.158387,-0.578169,-0.588276,0.261336,-0.596549,-0.708234,0.137634,1.463923,-0.231630,...,-0.561969,0.629936,0.108811,0.946626,0.173156,-1.529317,1.381716,-1.045786,0.052284,-0.534661
1,0.206775,-0.686740,-0.300016,0.621832,0.820380,0.300172,0.549845,-2.193271,1.718610,-0.285385,...,-2.226562,-0.245973,0.046079,1.256641,-0.375226,-1.497190,0.507080,0.975879,-1.505095,-0.208190
2,-1.032346,-0.800856,-1.273313,1.426241,-0.301278,0.359253,1.290237,-0.246600,1.377143,-0.759433,...,-1.109111,-0.229979,1.143392,1.716335,1.151444,-0.689020,-1.359141,0.408212,0.090199,0.001320
3,1.012362,0.180994,-0.006280,-0.572749,0.229795,-0.488464,0.185924,-0.255058,-0.336193,-0.153265,...,-1.729725,0.751417,-2.707426,-0.825416,-1.606554,2.177975,0.855611,-2.016360,0.187323,0.440859
4,0.684642,0.528635,0.287597,-0.302068,0.795122,-0.596539,0.846771,3.023128,-0.923668,0.790991,...,-0.601950,-1.058534,-2.032074,-0.399923,-0.787201,1.539175,-0.828353,-0.555788,-2.888968,1.493593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.815501,1.009798,0.480165,2.132632,-0.097618,-0.786746,0.468789,-1.419824,1.062300,1.876787,...,0.049447,-1.072751,0.855845,-0.605576,0.263048,0.603157,2.502607,-0.015364,0.407757,-0.836006
4996,0.993150,0.359608,1.332183,1.115888,2.004745,0.141506,1.276971,0.352938,-0.303591,1.069590,...,-1.336292,1.082891,0.825758,-0.894961,0.565565,-0.072382,1.526017,-1.009532,-1.024141,-0.392443
4997,-0.826997,0.224841,-1.138290,0.337155,0.065331,-0.057388,0.852314,0.033335,-0.362956,-0.899962,...,0.305421,0.676793,-1.383970,-0.533480,-0.618372,0.968560,0.787436,-1.927329,-0.222410,1.658867
4998,-0.412669,1.424471,-1.328137,-0.525071,0.090270,-1.870042,0.557969,-2.393077,0.480659,0.358168,...,0.852833,1.514963,-0.041692,-0.441971,-0.216937,0.272465,-0.594613,-0.773897,0.876928,1.706610


## Attempts without Mongo

In [4]:
%%timeit
frame.to_parquet("test.parquet", engine="pyarrow")
pd.read_parquet("test.parquet", engine="pyarrow")

33.2 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit
frame.to_pickle("test.pickle")
pd.read_pickle("test.pickle")

18.5 ms ± 3.47 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%%timeit
# note that here is no i/o to disk
x = Binary(pickle.dumps(frame))
pickle.loads(x)

1.87 ms ± 23.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Connecting to Mongo

In [7]:
MONGODB_SETTINGS = {'db': 'mongo', 'host': 'mongo'}

from mongoengine import *
connect(**MONGODB_SETTINGS)

MongoClient(host=['mongo:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

## Using Parquet, Convert frame to parquet and back

In [8]:
from antarctic.pandas_fields import ParquetFrameField

class Maffay(Document):
    frame = ParquetFrameField()

In [9]:
%%timeit
m=Maffay(frame=frame).save()
m.frame

36.6 ms ± 1.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Using Pickle, Convert using Pickle and back

In [ ]:
from antarctic.PandasFields import PicklePandasField

class Falco(Document):
    frame = PicklePandasField()